#### The code was run on a Kaggle notebook with 30GiB of RAM for combined 16+ hours in two sesssions 

In [ ]:

#! code for collecting (adjective, noun) pairs

import pandas as pd
import spacy

df = pd.read_parquet("./users.parquet", engine="fastparquet")
nlp = spacy.load("en_core_web_lg", disable=["ner"])
nlp.max_length = 20_000
print(nlp.max_length, *nlp.pipeline, sep="\n")

from collections import Counter
import pickle


def save_stuff(aspect_dict, upto="end"):
    for item in aspect_dict:
        aspect_dict[item]["bigrams"] = dict(
            Counter(aspect_dict[item]["bigrams"]).most_common()
        )

    with open(f"data_upto_{upto:>07}.pickle", "wb") as f:
        pickle.dump(aspect_dict, f)

aspect_dict = {}
sent_count = 0
bacth_size = 100_000
for review_index, review in enumerate(df["review/text"], start=0):
    item_dict = {
        "total_sentences": 0,
        "bigrams": {},
    }
    aspect_dict[review_index] = item_dict
    doc = nlp(review)
    for sentence in doc.sents:
        for token in sentence:
            if token.pos_ == "NOUN":
                for child in token.children:
                    if child.pos_ == "ADJ":
                        aspect = f"{child.lemma_} {token.lemma_}"
                        item_dict["bigrams"].setdefault(aspect, 0)
                        item_dict["bigrams"][aspect] += 1
        item_dict["total_sentences"] += 1
        sent_count += 1
    if review_index % bacth_size == 0:
        save_stuff(aspect_dict, upto=review_index)
    print(f"\rReview Number: {review_index} Sentences Processed: {sent_count}", end="")

save_stuff(aspect_dict, upto = len(df))

In [1]:
import pickle
with open("aspect_freq/final_users_aspects.pickle", "rb") as f:
    user_aspects = pickle.load(f)
with open("aspect_freq/final_prods_aspects.pickle", "rb") as f:
    prod_aspects = pickle.load(f)

In [3]:
for key, val in user_aspects.items():
    print(key)
    print(val)
    if key == 15:
        break

0
{'dark fruit': 2, 'heavy body': 1, 'longlasting head': 1, 'many ale': 1, 'other ale': 1, 'belgian ale': 1, 'dark ale': 1, 'strong ale': 1, 'large quantity': 1}
1
{'deep body': 1, 'dark body': 1, 'nice head': 1, 'foamy head': 1, 'thin cap': 1, 'tight cap': 1, 'nice lacing': 1, 'persistent lacing': 1, 'beautiful maltiness': 1, 'enough bite': 1, 'thin body': 1, 'dry finish': 1, 'hoppy finish': 1, 'crazy ale': 1}
2
{'nice color': 1, 'sweet fruitiness': 1}
3
{'massive head': 1, 'fizzy head': 1, 'next minute': 1, 'great bouquet': 1, 'good depth': 1, 'more chocolate': 1, 'drinkable interpretation': 1, 'black porter': 1}
4
{'orange colour': 1, 'lively nose': 1, 'light note': 1, 'interesting palate': 1, 'sweet note': 1}
5
{'thin cap': 1, 'moderate lacing': 1, 'perfect bitterness': 1, 'nice lot': 1, 'full lot': 1, 'flowery hop': 1, 'full mouthfeel': 1, 'excellent bite': 1, 'long finish': 1, 'dry finish': 1, 'golden color': 1, 'hazy color': 1, 'fresh hop': 1, 'only difference': 1, 'amazing arom

In [4]:
for key, val in prod_aspects.items():
    print(key)
    print(val)
    if key == 15:
        break

0
{'foamy head': 3, 'dark beer': 3, 'most beer': 3, 'good beer': 3, 'good carbonation': 2, 'clumpy lacing': 2, 'light vanilla': 2, 'nice beer': 2, 'fluffy head': 2, 'dark malt': 2, 'roasted malt': 2, 'artificial sweetener': 2, 'light body': 2, 'dark lager': 2, 'sweet liquorice': 2, 'deep amber': 1, 'dark amber': 1, 'white head': 1, 'sweet malt': 1, 'colored head': 1, 'gorgeous lacing': 1, 'opaque chocolate': 1, 'dark chocolate': 1, 'red hue': 1, 'smokey malt': 1, 'delicious t': 1, 'perfect carb': 1, 'dry finish': 1, 'huge fan': 1, 'good one': 1, 'tan head': 1, 'good retention': 1, 'standard lager': 1, 'pale lager': 1, 'mild coffee': 1, 'nuts hint': 1, 'mild taste': 1, 'main flavour': 1, 'light finish': 1, 'bitter finish': 1, 'heavy light': 1, 'frothy head': 1, 'thick head': 1, 'clear garnet': 1, 'dark garnet': 1, 'mild malt': 1, 'roasty malt': 1, 'little coffee': 1, 'much sweetness': 1, 'black lager': 1, 'short length': 1, 'easy beer': 1, 'beautiful color': 1, 'dark color': 1, 'brown c